In [10]:
import numpy as np
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import colorsys

def create_attention_demonstration(text, n_heads=4):
    """
    Demonstration of attention mechanism patterns in 3D space
    """
    # Limit tokens to reduce memory usage
    tokens = text.split()[:25]  # Limit to first 25 tokens
    n_tokens = len(tokens)
    
    np.random.seed(42)
    embeddings_high = np.random.randn(n_tokens, 128)  # Reduced dimensions
    
    # Simple positioning for memory efficiency
    angles = np.linspace(0, 2*np.pi, n_tokens, endpoint=False)
    positions_3d = np.column_stack([
        np.cos(angles) * 3 + np.random.randn(n_tokens) * 0.5,
        np.sin(angles) * 3 + np.random.randn(n_tokens) * 0.5,
        np.random.randn(n_tokens) * 2
    ])
    
    attention_heads = []
    head_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    
    for h in range(n_heads):
        if h == 0:  # Local attention
            weights = np.exp(-np.abs(np.arange(n_tokens)[:, None] - np.arange(n_tokens)[None, :]) / 2)
        elif h == 1:  # Content words
            weights = np.random.beta(2, 3, (n_tokens, n_tokens))
        elif h == 2:  # Long range
            weights = np.random.gamma(1, 1, (n_tokens, n_tokens))
        else:  # Mixed
            weights = np.random.exponential(1, (n_tokens, n_tokens))
        
        weights = np.exp(weights) / np.exp(weights).sum(axis=-1, keepdims=True)
        attention_heads.append(weights)
    
    fig = go.Figure()
    
    # Reduce number of connections for performance
    for head_idx in range(n_heads):
        attention_weights = attention_heads[head_idx]
        color = head_colors[head_idx]
        
        # Only show top connections
        top_indices = np.unravel_index(np.argsort(-attention_weights.flatten())[:n_tokens*2], attention_weights.shape)
        
        for idx in range(len(top_indices[0])):
            i, j = top_indices[0][idx], top_indices[1][idx]
            if i != j:
                weight = attention_weights[i, j]
                
                # Simple curved line
                t = np.linspace(0, 1, 15)  # Fewer points
                mid_z = (positions_3d[i, 2] + positions_3d[j, 2]) / 2 + weight * 2
                
                curve_x = positions_3d[i, 0] * (1-t) + positions_3d[j, 0] * t
                curve_y = positions_3d[i, 1] * (1-t) + positions_3d[j, 1] * t
                curve_z = positions_3d[i, 2] * (1-t) + positions_3d[j, 2] * t + 2 * weight * np.sin(np.pi * t)
                
                fig.add_trace(go.Scatter3d(
                    x=curve_x, y=curve_y, z=curve_z,
                    mode='lines',
                    line=dict(color=color, width=max(2, weight * 8)),
                    opacity=min(weight * 3, 0.7),
                    hovertemplate=f'{tokens[i]} → {tokens[j]}<extra></extra>',
                    showlegend=False
                ))
    
    # Add nodes
    fig.add_trace(go.Scatter3d(
        x=positions_3d[:, 0], y=positions_3d[:, 1], z=positions_3d[:, 2],
        mode='markers+text',
        text=tokens,
        textposition='middle center',
        marker=dict(size=12, color='gold', line=dict(color='black', width=1)),
        textfont=dict(size=10, color='white'),
        showlegend=False
    ))
    
    # Simple legend
    head_names = ['Local', 'Semantic', 'Long-range', 'Mixed']
    for i, name in enumerate(head_names):
        fig.add_trace(go.Scatter3d(
            x=[None], y=[None], z=[None],
            mode='lines',
            line=dict(color=head_colors[i], width=4),
            name=name,
            showlegend=True
        ))
    
    fig.update_layout(
        title='Attention Mechanism Demonstration',
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False), 
            zaxis=dict(visible=False),
            bgcolor='rgb(15, 15, 25)'
        ),
        paper_bgcolor='rgb(15, 15, 25)',
        font=dict(color='white'),
        height=600,
        showlegend=True
    )
    
    return fig

# Short paragraph
text = "Natural language processing enables machines to understand human language. Transformers use attention mechanisms to identify word relationships. These models process context dependencies efficiently."

fig = create_attention_demonstration(text)
fig.show()